In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sne
import IPython.display as display
import calendar
from imblearn.over_sampling import SMOTE
import sklearn.linear_model as linear_model
from scipy.sparse import hstack
from scipy.sparse import vstack
from scipy.sparse import coo_matrix
from sklearn.feature_extraction import FeatureHasher
from __future__ import division
from collections import Counter
from sklearn.externals import joblib
%matplotlib inline

In [3]:
train_coloums=['click','hour','C1','banner_pos','site_category','site_domain'
 ,'app_domain','app_category','device_type','device_conn_type'
,'device_model','C14','C15','C16','C17','C18','C19','C20','C21']

In [3]:
chunlsize=500000
train_data_chunks=pd.read_csv('/home/ashwin/Downloads/clickthroughrate/train.csv',nrows=28000000
                 ,chunksize=chunlsize,usecols=train_coloums,dtype={'click':np.int8,'hour':np.str,'C1':np.int16,
                        'banner_pos':np.int8,'site_category':np.str,
                        'site_domain':np.str,'app_domain':np.str,
                        'app_category':np.str,'device_type':np.int8,'device_conn_type':np.int8,'C14':np.int16,'C15':np.int16,'C16':np.int16,
'C17':np.int16,'C18':np.int8,'C19':np.int16,'C20':np.int32,'C21':np.int16})
#df = pd.concat([chunk for chunk in data_chunks])

In [4]:
#list(df)
#df.dtypes
#df.memory_usage()
#a=df['site_domain'].head()
#print df['site_domain'].head()
#a=np.int8(a)
#print a
#Counter(df['C17']).most_common()
#df['device_type'].value_counts()
#pd.get_dummies(df['device_type'].head(150000))
#df.drop(df[df.device_type == 2].index, inplace=True)

In [5]:
model=clf = linear_model.SGDClassifier(penalty='l1',n_jobs=-1
                                       ,random_state=100,
                                       class_weight={0:.17, 1:.83})

/home/ashwin/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [2]:
#data3=hstack((data3,data4.as_matrix()))
def conversion_hashing(data,n):
    featureHasher=FeatureHasher(n_features=n,input_type='string')
    data_converted=featureHasher.fit_transform(data.astype('str'))
    return data_converted

In [8]:
i=1
for chunk in train_data_chunks:
    chunk["day"] = chunk["hour"].map(lambda v: str(str(v)[-4:-2]))
    chunk['year']=chunk["hour"].map(lambda v: str('20'+str(v)[0:2]))
    chunk['month']=chunk["hour"].map(lambda v: int(str(v)[2:4]))
    #chunk['month']=chunk['month'].apply(lambda x: calendar.month_abbr[x])
    chunk["hour"] = chunk["hour"].map(lambda v: str(str(v)[-2:]))
    click= chunk['click']
    chunk.drop(['click'],axis=1,inplace=True)
    
    dd=(chunk[['day','hour','year','month']]).as_matrix()
    dd=hstack([dd.astype(np.int16),coo_matrix(pd.get_dummies(chunk[['banner_pos','device_type','device_conn_type',
              'C15','C16','C18']]).as_matrix())])
    dd=hstack([dd.astype(np.int16),coo_matrix(conversion_hashing(chunk['device_model'],5))])
    dd=hstack([dd.astype(np.int16),coo_matrix(conversion_hashing(chunk['site_category'],5))])
    dd=hstack([dd.astype(np.int16),coo_matrix(conversion_hashing(chunk['site_domain'],5))])
    dd=hstack([dd.astype(np.int16),coo_matrix(conversion_hashing(chunk['app_category'],5))])
    dd=hstack([dd.astype(np.int16),coo_matrix(conversion_hashing(chunk['app_domain'],5))])
    dd=hstack([dd.astype(np.int16),coo_matrix(conversion_hashing(chunk['C14'],8))])
    dd=hstack([dd.astype(np.int16),coo_matrix(conversion_hashing(chunk['C17'],8))])
    dd=hstack([dd.astype(np.int16),coo_matrix(conversion_hashing(chunk['C19'],5))])
    dd=hstack([dd.astype(np.int16),coo_matrix(conversion_hashing(chunk['C20'],5))])
    dd=hstack([dd.astype(np.int16),coo_matrix(conversion_hashing(chunk['C21'],5))])
    dd=hstack([dd.astype(np.int16),coo_matrix(conversion_hashing(chunk['C1'],4))])
    del chunk
    if(i==1):
        model.partial_fit(dd,click,classes=np.unique(click))
        i=0
    else:
        model.partial_fit(dd,click)

In [9]:
# Dump the trained decision tree classifier with Pickle
#SGD_pkl_filename = 'SGD_pickle.pkl'
# Open the file to save as pkl file
#SGD_Model_model_pkl = open(SGD_pkl_filename, 'wb')
#pickle.dump(model, SGD_Model_model_pkl)
# Close the pickle instances
#SGD_Model_model_pkl.close()

#df['click'].value_counts().plot(kind='bar')
#df['click'].mean()*100
joblib.dump(model, 'SGD.pkl') 

['SGD.pkl']

In [3]:
model1 = joblib.load('SGD.pkl') 

In [4]:

validation_coloums=['click','hour','C1','banner_pos','site_category','site_domain'
 ,'app_domain','app_category','device_type','device_conn_type'
,'device_model','C14','C15','C16','C17','C18','C19','C20','C21']

In [5]:
chunlsize=500000
validation_data=pd.read_csv('/home/ashwin/Downloads/clickthroughrate/train.csv',skiprows=range(1,28000000),
                 chunksize=chunlsize,usecols=validation_coloums,dtype={'click':np.int8,'hour':np.str,'C1':np.int16,
                        'banner_pos':np.int8,'site_category':np.str,
                        'site_domain':np.str,'app_domain':np.str,
                        'app_category':np.str,'device_type':np.int8,'device_conn_type':np.int8
                        ,'C14':np.int16,'C15':np.int16,'C16':np.int16,'C17':np.int16,'C18':np.int8,
                        'C19':np.int16,'C20':np.int32,'C21':np.int16})
#df = pd.concat([chunk for chunk in data_chunks])

In [6]:
scores=[]
for chunk in validation_data:
    chunk["day"] = chunk["hour"].map(lambda v: str(str(v)[-4:-2]))
    chunk['year']=chunk["hour"].map(lambda v: str('20'+str(v)[0:2]))
    chunk['month']=chunk["hour"].map(lambda v: int(str(v)[2:4]))
    #chunk['month']=chunk['month'].apply(lambda x: calendar.month_abbr[x])
    chunk["hour"] = chunk["hour"].map(lambda v: str(str(v)[-2:]))
    click= chunk['click']
    chunk.drop(['click'],axis=1,inplace=True)
    
    dd=(chunk[['day','hour','year','month']]).as_matrix()
    dd=hstack([dd.astype(np.int16),coo_matrix(pd.get_dummies(chunk[['banner_pos','device_type','device_conn_type',
              'C15','C16','C18']]).as_matrix())])
    dd=hstack([dd.astype(np.int16),coo_matrix(conversion_hashing(chunk['device_model'],5))])
    dd=hstack([dd.astype(np.int16),coo_matrix(conversion_hashing(chunk['site_category'],5))])
    dd=hstack([dd.astype(np.int16),coo_matrix(conversion_hashing(chunk['site_domain'],5))])
    dd=hstack([dd.astype(np.int16),coo_matrix(conversion_hashing(chunk['app_category'],5))])
    dd=hstack([dd.astype(np.int16),coo_matrix(conversion_hashing(chunk['app_domain'],5))])
    dd=hstack([dd.astype(np.int16),coo_matrix(conversion_hashing(chunk['C14'],8))])
    dd=hstack([dd.astype(np.int16),coo_matrix(conversion_hashing(chunk['C17'],8))])
    dd=hstack([dd.astype(np.int16),coo_matrix(conversion_hashing(chunk['C19'],5))])
    dd=hstack([dd.astype(np.int16),coo_matrix(conversion_hashing(chunk['C20'],5))])
    dd=hstack([dd.astype(np.int16),coo_matrix(conversion_hashing(chunk['C21'],5))])
    dd=hstack([dd.astype(np.int16),coo_matrix(conversion_hashing(chunk['C1'],4))])
    del chunk
    score=model.score( dd , click_valid )
    scores.append(score)
    #if(i==1):
     #   dd1=coo_matrix(dd).astype(np.int16)
      #  i=0
    #else:
     #   dd1=vstack([(coo_matrix(dd1.tocsr())).astype(np.int16)
     #              ,(coo_matrix(dd.tocsr())).astype(np.int16)])

In [ ]:
print(sum(scores)/float(len(scores)))